# 第4章: データと指示の分離

- [レッスン](#lesson)
- [演習](#exercises)
- [サンプル練習場](#example-playground)

## セットアップ

以下のセットアップセルを実行して、APIキーを読み込み、`get_completion`ヘルパー関数を設定してください。

In [ ]:
!pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

---

## レッスン

多くの場合、私たちは完全なプロンプトを書きたくないのですが、代わりに**Claude に送信する前に追加の入力データで後から修正できるプロンプトテンプレート**が欲しいものです。これは、Claude に毎回同じことをしてもらいたいが、Claude がそのタスクで使用するデータが毎回異なる場合に便利です。

幸いなことに、**プロンプトの固定骨格を可変ユーザー入力から分離し、完全なプロンプトを Claude に送信する前にユーザー入力をプロンプトに置換する**ことで、これを非常に簡単に行うことができます。

以下では、置換可能なプロンプトテンプレートの書き方と、ユーザー入力の置換方法について、ステップバイステップで説明します。

### 例

この最初の例では、Claude に動物の鳴き声ジェネレーターとして行動するよう求めています。Claude に送信される完全なプロンプトは、入力（この場合は「Cow」）で置換された`PROMPT_TEMPLATE`だけであることに注目してください。完全なプロンプトを印刷する際に、f-stringによって「Cow」という単語が`ANIMAL`プレースホルダーを置き換えることに注意してください。

**注記：** 実際には、プレースホルダー変数を特定の名前で呼ぶ必要はありません。この例では`ANIMAL`と呼びましたが、`CREATURE`や`A`と同じように呼ぶこともできました（ただし、一般的には変数名を具体的で関連性のあるものにすることが良いです。そうすることで、置換なしでもプロンプトテンプレートが理解しやすくなり、ユーザーの解析可能性のためです）。変数に名前を付けたものが、プロンプトテンプレートのf-stringで使用するものであることを確認してください。

In [ ]:
# Variable content
ANIMAL = "Cow"

# Prompt template with a placeholder for the variable content
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

なぜこのように入力を分離して置換したいのでしょうか？**プロンプトテンプレートは反復的なタスクを簡素化します**。プロンプトにコンテンツを送信する第三者ユーザーを招待するプロンプト構造を構築するとします（この場合、音を生成したい動物）。これらの第三者ユーザーは、完全なプロンプトを書いたり見たりする必要さえありません。彼らは変数を埋めるだけです。

ここでは変数とf-stringを使用してこの置換を行っていますが、format()メソッドでも実行できます。

**注記：** プロンプトテンプレートには、必要な数だけ変数を含めることができます！

このような置換変数を導入するとき、**Claude が変数の開始と終了の場所を知っている**ことを確認することが非常に重要です（指示やタスクの説明と対比して）。指示と置換変数の間に分離がない例を見てみましょう。

私たちの人間の目には、下記のプロンプトテンプレートで変数がどこで始まり、どこで終わるかは非常に明確です。しかし、完全に置換されたプロンプトでは、その境界が不明確になります。

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. {EMAIL} <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

ここで、**Claude は「Yo Claude」を書き直すべきメールの一部だと思っています**！これは「Dear Claude」で書き直しを始めていることからわかります。人間の目には、特にメールがどこで始まり、どこで終わるかのプロンプトテンプレートでは明確ですが、置換後のプロンプトではずっと不明確になります。

どうやってこれを解決するでしょうか？**入力をXMLタグで囲みます**！下記でこれを行い、ご覧のとおり、出力にもう「Dear Claude」はありません。

[XMLタグ](https://docs.anthropic.com/claude/docs/use-xml-tags)は`<tag></tag>`のような角括弧タグです。それらはペアになっており、`<tag>`のような開始タグと、`</tag>`のような`/`でマークされた閉じタグから構成されます。XMLタグはコンテンツを囲むために使用されます。例：`<tag>content</tag>`。

**注記：** Claude は幅広い区切り文字や分離符を認識し、使用することができますが、**Claude 用の区切り文字として特にXMLタグを使用することを推奨します**。Claude はXMLタグをプロンプト整理メカニズムとして認識するように特別に訓練されているからです。関数呼び出し以外では、**パフォーマンスを最大限に向上させるためにClaude が訓練された特別なソースXMLタグはありません**。私たちは意図的にClaude をこのように非常に柔軟でカスタマイズ可能にしました。

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. <email>{EMAIL}</email> <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

XMLタグがどのように私たちを助けることができるかの別の例を見てみましょう。

以下のプロンプトでは、**Claude がプロンプトのどの部分が指示なのか、どの部分が入力なのかを間違って解釈しています**。フォーマットのために`Each is about an animal, like rabbits`をリストの一部だと間違って考えています。ユーザー（`SENTENCES`変数を埋める人）はおそらくそれを望んでいませんでした。

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

これを修正するために、**ユーザー入力の文をXMLタグで囲む**だけです。これにより、`Each is about an animal, like rabbits.`の前の誤解を招くハイフンにもかかわらず、入力データがどこで始まり、どこで終わるかをClaude に示します。

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f""" Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
{SENTENCES}
</sentences>"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

**注記：** 「Each is about an animal」プロンプトの間違ったバージョンでは、この例でClaude に私たちが望む方法で間違って応答してもらうために、ハイフンを含める必要がありました。これはプロンプティングについての重要な教訓です：**小さな詳細が重要です**！**プロンプトのタイプミスや文法エラーをチェックすることは**常に価値があります。Claude はパターンに敏感で（ファインチューニング前の初期には、生のテキスト予測ツールでした）、あなたがミスをするとミスをしやすく、あなたが賢く聞こえると賢くなり、あなたがバカげて聞こえるとよりバカげるようになります。

上記のコンテンツを変更せずにレッスンのプロンプトを試したい場合は、レッスンノートブックの一番下までスクロールして[**サンプル練習場**](#example-playground)にアクセスしてください。

---

## 演習
- [演習 4.1 - 俳句のトピック](#exercise-41---haiku-topic)
- [演習 4.2 - タイプミスのある犬の質問](#exercise-42---dog-question-with-typos)
- [演習 4.3 - 犬の質問パート2](#exercise-42---dog-question-part-2)

### 演習 4.1 - 俳句のトピック
`PROMPT`を修正して、`TOPIC`という変数を受け取り、そのトピックについての俳句を出力するテンプレートにしてください。この演習は、f-stringを使った変数テンプレート構造の理解をテストすることを目的としています。

In [ ]:
# Variable content
TOPIC = "Pigs"

# Prompt template with a placeholder for the variable content
PROMPT = f""

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("pigs", text.lower()) and re.search("haiku", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが必要な場合は、下のセルを実行してください！

In [ ]:
from hints import exercise_4_1_hint; print(exercise_4_1_hint)

### 演習 4.2 - タイプミスのある犬の質問
XMLタグを追加して`PROMPT`を修正し、Claude が正しい答えを出すようにしてください。

プロンプトの他の部分はできるだけ変更しないようにしてください。乱雑でミスだらけの文章は意図的なものなので、Claude がそのようなミスにどう反応するかを確認できます。

In [ ]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが必要な場合は、下のセルを実行してください！

In [ ]:
from hints import exercise_4_2_hint; print(exercise_4_2_hint)

### 演習 4.3 - 犬の質問パート2
XMLタグを追加**せずに**`PROMPT`を修正してください。代わりに、プロンプトから1つか2つの単語だけを削除してください。

上記の演習と同様に、プロンプトの他の部分はできるだけ変更しないようにしてください。これにより、Claude がどのような言語を解析し理解できるかを示します。

In [ ]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが必要な場合は、下のセルを実行してください！

In [ ]:
from hints import exercise_4_3_hint; print(exercise_4_3_hint)

### おめでとうございます！

ここまでのすべての演習を解いた場合、次の章に進む準備ができています。楽しいプロンプティングを！

---

## サンプル練習場

これは、このレッスンで示されたプロンプトの例を自由に試し、プロンプトを調整してClaude の応答にどのような影響を与えるかを確認するためのエリアです。

In [ ]:
# Variable content
ANIMAL = "Cow"

# Prompt template with a placeholder for the variable content
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. {EMAIL} <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. <email>{EMAIL}</email> <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f""" Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
{SENTENCES}
</sentences>"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))